<a href="https://colab.research.google.com/github/antonsysoev/ktmi_pm/blob/main/KTMI_II_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%load_ext rpy2.ipython


The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [5]:
%%R

# read D bits from integer x:
binint=function(x,D)
{ 
    x=rev(intToBits(x)[1:D]) # get D bits
# remove extra 0s from raw type:
    as.numeric(unlist(strsplit(as.character(x),""))[(1:D)*2])
}

# convert binary vector into integer: code inspired in
# http://stackoverflow.com/questions/12892348/
# in-r-how-to-convert-binary-string-to-binary-or-decimal-value
intbin=function(x) sum(2^(which(rev(x==1))-1))
# sum a raw binary object x (evaluation function):
sumbin=function(x) sum(as.numeric(x))
# max sin of binary raw object x (evaluation function):
maxsin=function(x,Dim) sin(pi*(intbin(x))/(2^Dim))

In [6]:
%%R
# compute the bag factory profit for x:
# x - a vector of prices
profit=function(x) # x - a vector of prices
{   
    x=round(x,digits=0) # convert x into integer
    s=sales(x) # get the expected sales
    c=cost(s) # get the expected cost
    profit=sum(s*x-c) # compute the profit
    return(profit)
# local variables x, s, c and profit are lost from here
}
# compute the cost for producing units:
# units - number of units produced
# A - fixed cost, cpu - cost per unit
cost=function(units,A=100,cpu=35-5*(1:length(units)))
{ return(A+cpu*units) }
# compute the estimated sales for x:
# x - a vector of prices, m - marketing effort
# A, B, C - constants of the estimated function
sales=function(x,A=1000,B=200,C=141,
m=seq(2,length.out=length(x),by=-0.25))
{ return(round(m*(A/log(x+B)-C),digits=0))}
# example of a simple recursive function:
fact=function(x=0) # x - integer number
{ if(x==0) return(1) else return(x*fact(x-1))}  

In [7]:
%%R
sphere=function(x) sum(x^2)
rastrigin=function(x) 10*length(x)+sum(x^2-10*cos(2*pi*x))

In [11]:
%%R
install.packages("genalg")
library(genalg) # load genalg package
# genetic algorithm search for bag prices:


R[write to console]: Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

R[write to console]: trying URL 'https://cran.rstudio.com/src/contrib/genalg_0.2.0.tar.gz'

R[write to console]: Content type 'application/x-gzip'
R[write to console]:  length 7712 bytes

R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console

In [12]:
%%R
# genetic algorithm search for bag prices:
D=5 # dimension (number of prices)
MaxPrice=1000
Dim=ceiling(log(MaxPrice,2)) # size of each price (=10)
size=D*Dim # total number of bits (=50)
intbin=function(x) # convert binary to integer
{ sum(2^(which(rev(x==1))-1)) } # explained in Chapter 3
bintbin=function(x) # convert binary to D prices
{ # note: D and Dim need to be set outside this function
s=vector(length=D)
for(i in 1:D) # convert x into s:
{ ini=(i-1)*Dim+1;end=ini+Dim-1
s[i]=intbin(x[ini:end])
}
return(s)
}
bprofit=function(x) # profit for binary x
{ s=bintbin(x)
s=ifelse(s>MaxPrice,MaxPrice,s) # repair!
f=-profit(s) # minimization task!
return(f)
}
# genetic algorithm execution:
G=rbga.bin(size=size,popSize=50,iters=100,zeroToOneRatio=1,
evalFunc=bprofit,elitism=1)
# show results:
b=which.min(G$evaluations) # best individual
cat("best:",bintbin(G$population[b,]),"f:",-G$evaluations[b],
"\n")
pdf("genalg1.pdf") # personalized plot of G results
plot(-G$best,type="l",lwd=2,ylab="profit",xlab="generations")
lines(-G$mean,lty=2,lwd=2)
legend("bottomright",c("best","mean"),lty=1:2,lwd=2)
dev.off()
summary(G,echo=TRUE) # same as summary.rbga

best: 401 351 391 374 447 f: 43668 
GA Settings
  Type                  = binary chromosome
  Population size       = 50
  Number of Generations = 100
  Elitism               = 1
  Mutation Chance       = 0.0196078431372549

Search Domain
  Var 1 = [,]
  Var 0 = [,]

GA Results
  Best Solution : 0 1 1 0 0 1 0 0 0 1 0 1 0 1 0 1 1 1 1 1 0 1 1 0 0 0 0 1 1 1 0 1 0 1 1 1 0 1 1 0 0 1 1 0 1 1 1 1 1 1 


In [13]:
%%R
# evolutionary algorithm for sphere:
sphere=function(x) sum(x^2)
D=2
monitor=function(obj)
{ if(i==1)
{ plot(obj$population,xlim=c(-5.2,5.2),ylim=c(-5.2,5.2),
xlab="x1",ylab="x2",type="p",pch=16,
col=gray(1-i/maxit))
}
else if(i%%K==0) points(obj$population,pch=16,
col=gray(1-i/maxit))
i<<-i+1 # global update
}
maxit=100
K=5 # store population values every K generations
i=1 # initial generation
# evolutionary algorithm execution:
pdf("genalg2.pdf",width=5,height=5)
set.seed(12345) # set for replicability purposes
E=rbga(rep(-5.2,D),rep(5.2,D),popSize=5,iters=maxit,
monitorFunc=monitor,evalFunc=sphere)
b=which.min(E$evaluations) # best individual
cat("best:",E$population[b,],"f:",E$evaluations[b],"\n")
dev.off()

best: 0.02438428 0.09593309 f: 0.009797751 
png 
  2 


In [15]:
%%R
### sphere-DEoptim.R file ###
install.packages("DEoptim")
library(DEoptim) # load DEoptim
sphere=function(x) sum(x^2)
D=2
maxit=100
set.seed(12345) # set for replicability
C=DEoptim.control(strategy=1,NP=5,itermax=maxit,CR=0.9,F=0.8,
trace=25,storepopfrom=1,storepopfreq=1)
# perform the optimization:
D=suppressWarnings(DEoptim(sphere,rep(-5.2,D),rep(5.2,D),
control=C))
# show result:
summary(D)
pdf("DEoptim.pdf",onefile=FALSE,width=5,height=9,
colormodel="gray")
plot(D,plot.type="storepop")
dev.off()
cat("best:",D$optim$bestmem,"f:",D$optim$bestval,"\n")


R[write to console]: Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

R[write to console]: trying URL 'https://cran.rstudio.com/src/contrib/DEoptim_2.2-6.tar.gz'

R[write to console]: Content type 'application/x-gzip'
R[write to console]:  length 105635 bytes (103 KB)

R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[writ

Iteration: 25 bestvalit: 0.644692 bestmemit:    0.799515    0.073944
Iteration: 50 bestvalit: 0.308293 bestmemit:    0.550749   -0.070493
Iteration: 75 bestvalit: 0.290737 bestmemit:    0.535771   -0.060715
Iteration: 100 bestvalit: 0.256731 bestmemit:    0.504867   -0.042906

***** summary of DEoptim object ***** 
best member   :  0.50487 -0.04291 
best value    :  0.25673 
after         :  100 generations 
fn evaluated  :  202 times 
*************************************
best: 0.5048666 -0.0429055 f: 0.2567311 


In [17]:
%%R

### sphere-psoptim.R file ###
install.packages("pso")
library(pso) # load pso
sphere=function(x) sum(x^2)
D=2; maxit=10; s=5
set.seed(12345) # set for replicability
C=list(trace=1,maxit=maxit,REPORT=1,trace.stats=1,s=s)
# perform the optimization:
PSO=psoptim(rep(NA,D),fn=sphere,lower=rep(-5.2,D),
upper=rep(5.2,D),control=C)
# result:
pdf("psoptim1.pdf",width=5,height=5)
j=1 # j-th parameter
plot(xlim=c(1,maxit),rep(1,s),PSO$stats$x[[1]][j,],pch=19,
xlab="iterations",ylab=paste("s_",j," value",sep=""))
for(i in 2:maxit) points(rep(i,s),PSO$stats$x[[i]][j,],pch=19)
dev.off()
pdf("psoptim2.pdf",width=5,height=5)
plot(PSO$stats$error,type="l",lwd=2,xlab="iterations",
ylab="best fitness")
dev.off()
cat("best:",PSO$par,"f:",PSO$value,"\n")

R[write to console]: Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

R[write to console]: trying URL 'https://cran.rstudio.com/src/contrib/pso_1.0.3.tar.gz'

R[write to console]: Content type 'application/x-gzip'
R[write to console]:  length 13314 bytes (13 KB)

R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to c

best: 0.006601689 0.1126331 f: 0.0127298 


In [21]:
%%R
# four EDA types:
# adapted from (Gonzalez-Fernandez and Soto, 2012)
UMDA=CEDA(copula="indep",margin="norm"); UMDA@name="UMDA"
GCEDA=CEDA(copula="normal",margin="norm"); GCEDA@name="GCEDA"
CVEDA=VEDA(vine="CVine",indepTestSigLevel=0.01,
copulas = c("normal"),margin = "norm")
CVEDA@name="CVEDA"
DVEDA=VEDA(vine="DVine",indepTestSigLevel=0.01,
copulas = c("normal"),margin = "norm")
DVEDA@name="DVEDA"

R[write to console]: Error in CEDA(copula = "indep", margin = "norm") : 
  could not find function "CEDA"




Error in CEDA(copula = "indep", margin = "norm") : 
  could not find function "CEDA"


RInterpreterError: ignored

In [20]:
%%R


### sphere-EDA.R file ###
install.packages("copulaedas")
library(copulaedas)
sphere=function(x) sum(x^2)
D=2; maxit=10; LP=5
set.seed(12345) # set for replicability
# set termination criterion and report method:
setMethod("edaTerminate","EDA",edaTerminateMaxGen)
setMethod("edaReport","EDA",edaReportSimple)
# set EDA type:
UMDA=CEDA(copula="indep",margin="norm",popSize=LP,maxGen=maxit)
UMDA@name="UMDA (LP=5)"
# run the algorithm:
E=edaRun(UMDA,sphere,rep(-5.2,D),rep(5.2,D))
# show result:
show(E)
cat("best:",E@bestSol,"f:",E@bestEval,"\n")
# second EDA execution, using LP=100:
maxit=10; LP=100;
UMDA=CEDA(copula="indep",margin="norm",popSize=LP,maxGen=maxit)
UMDA@name="UMDA (LP=100)"
setMethod("edaReport","EDA",edaReportDumpPop) # pop_*.txt files
E=edaRun(UMDA,sphere,rep(-5.2,D),rep(5.2,D))
show(E)
cat("best:",E@bestSol,"f:",E@bestEval,"\n")
# read dumped files and create a plot:
pdf("eda1.pdf",width=7,height=7)
j=1; # j-th parameter
i=1;d=read.table(paste("pop_",i,".txt",sep=""))
plot(xlim=c(1,maxit),rep(1,LP),d[,j],pch=19,
xlab="iterations",ylab=paste("s_",j," value",sep=""))
for(i in 2:maxit)
{ d=read.table(paste("pop_",i,".txt",sep=""))
points(rep(i,LP),d[,j],pch=19)
}
dev.off()

R[write to console]: Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

R[write to console]: also installing the dependencies ‘gsl’, ‘copula’, ‘vines’


R[write to console]: trying URL 'https://cran.rstudio.com/src/contrib/gsl_2.1-7.tar.gz'

R[write to console]: Content type 'application/x-gzip'
R[write to console]:  length 213803 bytes (208 KB)

R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: 


Error in library(copulaedas) : there is no package called ‘copulaedas’


RInterpreterError: ignored